In [2]:
import sys
sys.path.append('../src/')

%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
from pathlib import Path
import geopandas as gpd
import hvplot.pandas
from shapely.geometry import Polygon
import hvplot
from ast import literal_eval

hvplot.extension('bokeh')

In [4]:
PATH = Path.cwd().parent.joinpath('data')

## Get selected vessels

See voyages of Sarah M and Ganado Express including weather

In [4]:
## Import vessels

s = pd.read_csv(PATH.joinpath('gfw_tracks', 'sarahm.csv'))
s['vessel'] = 'sarahm'
g = pd.read_csv(PATH.joinpath('gfw_tracks', 'ganadoexpress.csv'))
g['vessel'] = 'ganadoexpress'

df = pd.concat([s, g])
df.timestamp = pd.to_datetime(df.timestamp, unit='ms')
df = df[df.timestamp > '2023-11-01'].copy()
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(x=df.lon, y=df.lat), crs=4326)
len(gdf)

6393

In [12]:
# Import weather data

w = pd.read_csv(PATH.joinpath('meteo', 'selected_vessels.csv'))
w.timestamp = pd.to_datetime(w.timestamp, unit='ms')
w = w[(w.timestamp < '2023-11-20') & (w.timestamp > '2023-11-09')].copy()
cols = ['tempC', 'windspeedKmph']
for col in cols:
    w[col] = w[col].astype('int')
w = gpd.GeoDataFrame(w, geometry=gpd.points_from_xy(x=w.lon, y=w.lat), crs=4326)
len(w)

427

In [15]:
w.to_csv(PATH.joinpath('meteo', 'vessels_for_viz.csv'), index=False)

In [13]:

w[w.vessel=='ganadoexpress'].hvplot(geo=True, 
                             color='windspeedKmph', 
                             cmap='fire', 
                             width=800, 
                             height=600, 
                             hover_cols=['vessel', 'windspeedKmph', 'timestamp'],
                             tiles=True).opts(active_tools=['box_zoom', 'wheel_zoom']) 

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Longitude,Latitude]   (windspeedKmph,vessel,timestamp)

In [64]:
meteo = pd.read_csv(PATH.joinpath('meteo', 'meteo_results_hourly.csv'))
len(meteo)

32294

## Get events

In [16]:
events = []
for file in PATH.joinpath('gfw_data').glob('*-events-*'):
    name = file.stem
    df = pd.read_csv(file)
    df['vessel'] = name
    events.append(df)

df = pd.concat(events)
len(df)

# Clean
df.start = pd.to_datetime(df.start)
df.end = pd.to_datetime(df.end)
df['vessel_flag'] = df.vessel.apply(lambda x: x.split('(')[0][0:3])
df.vessel = df.vessel.apply(lambda x: x.split('-')[0][:-5])

In [17]:
# Get port visits

pv = df[df['type']=='port_visit'][['start','end','voyage','latitude','longitude',
                                   'portVisitName','portVisitFlag', 'vessel', 'vessel_flag']].copy()

In [18]:
# Most visited ports

pv.portVisitFlag.value_counts()[0:10]

portVisitFlag
AUS    1135
EGY     894
TUR     730
IDN     561
ROU     540
ESP     385
CHN     339
VNM     300
SGP     271
SAU     245
Name: count, dtype: int64

In [19]:
pv = gpd.GeoDataFrame(pv, geometry=gpd.points_from_xy(x=pv.longitude, y=pv.latitude), crs=4326)

In [21]:
ports = pv.groupby(['portVisitName']).agg(latitude=('latitude', 'median'),
                                          longitude=('longitude', 'median'),
                                          visits=('vessel', 'count'),
                                          vessels_diff=('vessel', lambda x: x.nunique())).reset_index()

ports = gpd.GeoDataFrame(ports, geometry=gpd.points_from_xy(x=ports.longitude, y=ports.latitude), crs=4326)

In [24]:
ports[['portVisitName', 'latitude', 'longitude', 'visits']].to_csv(PATH.joinpath('gfw_data', 'port_visits.csv'))

In [22]:
ports.hvplot(geo=True,
             tiles=True,
             width=800,
             height=600,
             size='visits'
             ).opts(active_tools=['wheel_zoom'])

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Longitude,Latitude]   (visits)

In [27]:
# How about the NL?

pvnl = pv[pv.portVisitFlag=='NLD'].copy()
len(pvnl)

115

In [108]:
pvnl.vessel.unique()

array(['SHORTHORN EXPRESS', 'ESPERANCE'], dtype=object)

There are only 115 port visits in the Netherlands. These are not all visits, because during the research I saw tracks going into the Netherlands ports that haven't been logged as a port visit by the GFW algorithms. I can use the tracks data and filter with port polygons.

In [92]:
# Create Polygons for ports

ijmuiden = ((4.504274, 52.493004),
          (4.910082, 52.449922),
          (4.89085588546282, 52.37956124860245),
          (4.394411, 52.413918),
          (4.504274, 52.493004))

rotterdam = ((3.892931,52.034854),
             (4.443900,51.956091),
             (4.423893,51.803771),
             (3.745890,51.899889),
             (3.892931,52.034854))

zeeland = ((3.299345,51.546999),
           (4.350941,51.467881),
           (4.273901,51.198332),
           (3.064373,51.403045),
           (3.299345,51.546999))

den_helder = ((4.641531,53.027905),
              (4.880262,53.010702),
              (4.892696,52.923835),
              (52.914838, 4.641531),
              (4.641531,53.027905))

urk = ((5.506152,52.730952),
       (5.690174,52.709111),
       (5.710574,52.598706),
       (5.535640,52.626007),
       (5.506152,52.730952)
       )

eemshaven = ((6.585831,53.652322),
             (7.044206,53.753703),
             (7.059318,53.266950),
             (6.578275,53.367753),
             (6.585831,53.652322))

harlingen = ((5.336631,53.214196),
             (5.525522,53.227767),
             (5.487744,53.129655),
             (5.319001,53.151561),
             (5.336631,53.214196))


In [103]:
# Import tracks

tracks = []

for file in PATH.joinpath('gfw_tracks').glob('*.csv'):
    name = file.stem
    df = pd.read_csv(file)
    df['vessel'] = name
    tracks.append(df)

tracks = pd.concat(tracks)
tracks = gpd.GeoDataFrame(tracks, geometry=gpd.points_from_xy(x=tracks.lon, y=tracks.lat), crs=4326)
len(tracks)

4643101

In [104]:
visits = []

cols = [ijmuiden, rotterdam, zeeland]
for col in cols:
    polygon = Polygon(col)
    gdf = tracks.loc[tracks.within(polygon)]
    visits.append(gdf)

portsnl = pd.concat(visits)
portsnl.seg_id = portsnl.seg_id.str[10:]
portsnl.seg_id = pd.to_datetime(portsnl.seg_id).dt.date

In [105]:
visits = portsnl.groupby(['vessel', 'seg_id']).agg(observations=('vessel', 'count')).reset_index()
visits[visits.seg_id.astype('str').str.contains('2023')].sort_values(by='seg_id', ascending=False).vessel.unique()

In [118]:
portsnl[portsnl.vessel=='omegastar'].hvplot(geo=True,
                                            tiles=True,
                                            hover_cols=['timestamp'],
                                            width=800,
                                            height=600)

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Longitude,Latitude]   (timestamp)

In [113]:
ports = pd.read_csv(PATH.joinpath('gfw_api', 'port_visits.csv'))
ports = gpd.GeoDataFrame(ports, geometry=gpd.points_from_xy(x=ports.lon, y=ports.lat), crs=4326)
cols = ['start', 'end']
for col in cols:
    ports[col] = pd.to_datetime(ports[col])

ports.sort_values(by=['vessel_name', 'start'], inplace=True)
len(ports)

7437

In [ ]:
# Calculate time between port visits

ports = ports.join(ports.groupby('vessel_id').shift(-1), rsuffix='-')
ports.rename(columns={'start-': 'start_next'}, inplace=True)
cols = [x for x in ports.columns if '-' not in x]
ports = ports[cols]
ports['journey'] = (ports.start_next- ports.end) / pd.to_timedelta(1, unit='D')

In [ ]:
miny = 51.23
maxy = 53.77
minx = 2.9
maxx = 7.07

ports[(ports.lat > miny) & (ports.lat < maxy) & (ports.lon > minx) & (ports.lon < maxx)].mmsi.unique()

In [ ]:
# Plot a histogram

ports[(ports.journey < 50) & (ports.journey > 5)].hvplot.hist(y='journey')

In [ ]:
ports[ports.journey > 20].vessel_name.value_counts()

## Loitering 

In [ ]:
loitering = pd.read_csv(PATH.joinpath('gfw_api', 'loitering.csv'))
loitering = gpd.GeoDataFrame(loitering, geometry=gpd.points_from_xy(x=loitering.lon, y=loitering.lat), crs=4326)
len(loitering)

In [ ]:
# Get histogram

loitering.hvplot.hist(y='loitering_hours')

In [ ]:
# Remove outliers

loitering = loitering[(loitering.loitering_hours < 500) & (loitering.loitering_hours > 12)].copy()
loitering.hvplot.hist(y='loitering_hours')

In [ ]:
loitering.explore()

In [ ]:
# Top 10 companies with most loitering hours

top10 = loitering.groupby('vessel_name').agg(hours = ('loitering_hours', 'sum')).sort_values(by='hours', ascending=False).reset_index()[0:10]
top10



In [ ]:
# Where did they loiter?

vessels = list(top10.vessel_name)
loitering[loitering.vessel_name.isin(vessels)].explore(column='loitering_hours',
                                                       cmap='fire')

Some observations:
1. The loitering events are concentrated in the Mediterenean (Spain, Middle East and around South Africa and Madagascar)
2. On the Atlantic the Bashar One Transport seems to stop. Often not that long, but still. 
3. Long loitering events are clustered around Crete. Also near Dominica, Bolivia and Mauritius. Why there?

In [ ]:
records = []
count = 0
with open(PATH.joinpath('meteo', 'meteo_results.json'), 'r') as file:
    for line in file:
        records.append(literal_eval(line))
        count += 1
        if count > 10:
            break

## Simulation

In [5]:
import numpy as np

In [67]:
# Get stream data

npz = np.load(PATH.joinpath('meteo', 'koe_fwd_2024-02-08_11.npz'))

In [68]:
longitude = npz['arr_1']
latitude = npz['arr_2']
timestamp = npz['arr_3']
vessels = npz['arr_4']

In [70]:
# Parse npz file

records = []
for lon, lat, time, vessel in zip(longitude, latitude, timestamp, vessels):
    for lo, la, t in zip(lon, lat, time):
        records.append({'vessel': vessel,
                        'longitude': lo,
                        'latitude': la,
                        'timestamp': t})
        
df = pd.DataFrame(records)
df = df[df.longitude.notna()]

# Write to file
df.to_csv(PATH.joinpath('meteo', 'simulation.csv'))